---
title: "Non-Standard Evaluation in R with rlang"
author: "Your Name"
date: "2023-11-14"
format:
 html:
    code-fold: show
    code-tools: true
    code-overflow: wrap
    code-block-bg: "#f1f3f5"
    code-block-border-left: "#31BAE9"
    highlight-style: github
execute:
  eval: false
jupyter: python3
---


# Non-Standard Evaluation in R with `rlang`

## Introduction

Non-standard evaluation (NSE) is a powerful feature in R that allows you to manipulate code expressions instead of just their evaluated values. This is particularly useful when writing functions that interact with variable names or expressions, such as data manipulation functions in the tidyverse. The `rlang` package provides modern tools for NSE, commonly referred to as **tidy evaluation**, which make writing such functions safer and more intuitive.

This document explains non-standard evaluation in R, focusing on the `rlang` package and its applications within the tidyverse ecosystem.

---

## Table of Contents
1. [Understanding Non-Standard Evaluation](#understanding-non-standard-evaluation)
2. [Limitations of Base R NSE Functions](#limitations-of-base-r-nse-functions)
3. [Tidy Evaluation with `rlang`](#tidy-evaluation-with-rlang)
    - [Capturing Symbols and Expressions](#capturing-symbols-and-expressions)
    - [Unquoting and Injecting Expressions](#unquoting-and-injecting-expressions)
    - [The Curly-Curly `{{ }}` Operator](#the-curly-curly--operator)
4. [Practical Examples](#practical-examples)
    - [Writing Functions with Tidy Evaluation](#writing-functions-with-tidy-evaluation)
    - [Handling Strings and Symbols Flexibly](#handling-strings-and-symbols-flexibly)
5. [Key Takeaways](#key-takeaways)
6. [Conclusion](#conclusion)

---


## Understanding Non-Standard Evaluation

In **standard evaluation**, function arguments are evaluated in the environment where the function is called before being passed to the function.


```{r}
x <- 10
standard_eval_example <- function(arg) {
  print(arg)
}

standard_eval_example(x)
# Output: 10
```

    summarize(count = n())
}

# Using the function directly works as expected
summarize_by(mtcars, cyl)
```

**Now, you want to call `summarize_by()` from within another function:**

```r
first_function <- function(data, group_col) {
  summarize_by(data, group_col)
}

# Attempting to use the function
first_function(mtcars, cyl)
```

**Issue:**

When you run `first_function(mtcars, cyl)`, you may encounter an error or unexpected behavior. This happens because the NSE mechanics in `summarize_by()` are trying to capture `group_col` from the environment of `summarize_by()`, not from the original caller.

**Solution:**

To correctly pass NSE arguments through multiple function calls, you need to explicitly capture and forward the expressions using `enquo()` and the unquoting operator `!!`.

**Corrected Code:**

```r
library(dplyr)

# The original function using NSE
summarize_by <- function(data, group_col) {
  data %>%
    group_by({{ group_col }}) %>%
    summarize(count = n())
}

# The wrapper function that properly handles NSE argument
first_function <- function(data, group_col) {
  # Capture the expression as a quosure
  group_col_quo <- enquo(group_col)
  # Pass the quosure to the inner function using unquoting
  summarize_by(data, !!group_col_quo)
}

# Using the function now works as expected
first_function(mtcars, cyl)
```

**Explanation:**

- **`enquo(group_col)`**: Captures the `group_col` argument as a **quosure**, which includes both the expression and its environment.
- **`!!group_col_quo`**: Unquotes the quosure when passing it to `summarize_by()`, ensuring that the original expression is evaluated in the correct context.

**Why This Works:**

- The `enquo()` function captures the argument provided to `first_function()` without evaluating it.
- By unquoting `!!group_col_quo` when calling `summarize_by()`, you effectively pass the captured expression so that `summarize_by()` can evaluate it properly with its NSE mechanics.

**Alternative with `...`:**

If you want to pass multiple NSE arguments or make your functions more flexible, you can use the `...` (ellipsis) to pass arguments along.

**Example:**

```r
summarize_by <- function(data, ...) {
  data %>%
    group_by(...) %>%
    summarize(count = n())
}

first_function <- function(data, ...) {
  summarize_by(data, ...)
}

# Using the function with multiple grouping variables
first_function(mtcars, cyl, gear)
```

**Note:**

- When using `...`, you can pass any number of NSE arguments, and they will be correctly handled by the inner function.
- This approach is suitable when you don't need to manipulate the NSE arguments inside the wrapper function.

---

**Practical Guidelines for Passing NSE Arguments:**

1. **Capture NSE Arguments with `enquo()` or `enquos()`:**

   - Use `enquo()` for a single argument.
   - Use `enquos()` for multiple arguments.

   ```r
   first_function <- function(data, group_col) {
     group_col_quo <- enquo(group_col)
     # ...
   }
   ```

2. **Forward Captured Quosures with Unquoting `!!` or Splicing `!!!`:**

   - Use `!!` to unquote a single quosure.
   - Use `!!!` to splice a list of quosures.

   ```r
   summarize_by(data, !!group_col_quo)
   ```

3. **Use `{{ }}` in the Innermost Function Only:**

   - The `{{ }}` operator is syntactic sugar for capturing and unquoting in a single step.
   - It works best when used directly within the function where the NSE argument originates.
   - When wrapping such functions, prefer explicit capturing and unquoting.

4. **Be Mindful of Environments:**

   - Quosures capture the environment, ensuring that variables are evaluated in the correct context.
   - This is crucial when NSE functions are nested within other functions.

---

**Complete Example in the Document:**

```r
library(dplyr)

# Inner function using NSE with {{ }}
summarize_by <- function(data, group_col) {
  data %>%
    group_by({{ group_col }}) %>%
    summarize(count = n())
}

# Wrapper function that properly handles NSE arguments
first_function <- function(data, group_col) {
  # Capture the NSE argument as a quosure
  group_col_quo <- enquo(group_col)
  # Pass it to the inner function using unquoting
  summarize_by(data, !!group_col_quo)
}

# Using the function
result <- first_function(mtcars, cyl)
print(result)
```

**Output:**

```
# A tibble: 3 × 2
    cyl count
  <dbl> <int>
1     4    11
2     6     7
3     8    14
```

---

**Explanation in the Context of the Document:**

### Handling NSE in Nested Function Calls

When you have functions that use tidy evaluation and you want to call them from within other functions, it's important to manage the NSE arguments properly. Here’s how you can do it:

1. **Capture the Argument in the Outer Function:**

   Use `enquo()` in the outer function to capture the NSE argument as a quosure.

   ```r
   first_function <- function(data, group_col) {
     group_col_quo <- enquo(group_col)
     # ...
   }
   ```

2. **Unquote the Argument in the Inner Function Call:**

   When calling the inner function, unquote the quosure using `!!` so that the inner function receives the correct expression.

   ```r
   summarize_by(data, !!group_col_quo)
   ```

3. **Use `{{ }}` in the Inner Function:**

   The inner function can use `{{ }}` to simplify the NSE handling.

   ```r
   summarize_by <- function(data, group_col) {
     data %>%
       group_by({{ group_col }}) %>%
       summarize(count = n())
   }
   ```

**Key Points:**

- **Quosures**: Quosures are expressions that carry their environment, allowing them to be evaluated correctly later on.
- **Capturing vs. Evaluating**: By capturing the argument in the outer function and unquoting it in the inner function, you maintain the correct evaluation context.
- **Avoid Double Evaluation**: Properly managing the NSE arguments prevents them from being accidentally evaluated too early or too late.

---

**Alternative Approach Using `enquos()` and `!!!`:**

If your inner function accepts multiple NSE arguments, you can use `enquos()` and the splicing operator `!!!`.

**Example:**

```r
library(dplyr)

# Inner function that groups by multiple columns
summarize_by <- function(data, ...) {
  data %>%
    group_by(...) %>%
    summarize(count = n())
}

# Wrapper function that captures multiple NSE arguments
first_function <- function(data, ...) {
  group_cols_quos <- enquos(...)
  summarize_by(data, !!!group_cols_quos)
}

# Using the function with multiple grouping columns
result <- first_function(mtcars, cyl, gear)
print(result)
```

**Output:**

```
# A tibble: 6 × 3
    cyl  gear count
  <dbl> <dbl> <int>
1     4     3     1
2     4     4     8
3     4     5     2
4     6     3     2
5     6     4     4
6     8     3    13
```

---

**Conclusion:**

When dealing with NSE and nested function calls in R, it's crucial to:

- Properly capture NSE arguments using `enquo()` or `enquos()`.
- Forward these arguments using unquoting (`!!`) or splicing (`!!!`).
- Understand that `{{ }}` is a convenient shortcut for simple cases but may require explicit handling in more complex scenarios.

By following these practices, you ensure that your functions behave as expected, making your code robust and reliable.

---

**Updated Table of Contents:**

1. [Understanding Non-Standard Evaluation](#understanding-non-standard-evaluation)
2. [Limitations of Base R NSE Functions](#limitations-of-base-r-nse-functions)
3. [Tidy Evaluation with `rlang`](#tidy-evaluation-with-rlang)
    - [Capturing Symbols and Expressions](#capturing-symbols-and-expressions)
    - [Unquoting and Injecting Expressions](#unquoting-and-injecting-expressions)
    - [The Curly-Curly `{{ }}` Operator](#the-curly-curly--operator)
4. [Practical Examples](#practical-examples)
    - [Writing Functions with Tidy Evaluation](#writing-functions-with-tidy-evaluation)
    - [Handling Strings and Symbols Flexibly](#handling-strings-and-symbols-flexibly)
    - [Calling Functions that Use NSE from Within Other Functions](#calling-functions-that-use-nse-from-within-other-functions)
5. [Key Takeaways](#key-takeaways)
6. [Conclusion](#conclusion)

---

Feel free to include this explanation in your document to provide a comprehensive guide on how to handle non-standard evaluation in R, especially when dealing with nested function calls within the tidyverse ecosystem.




## Handling `tar_read()` and Non-Standard Evaluation in Functions

When working with the `targets` package in R, you might encounter situations where you want your functions to accept either the name of a target (as a symbol or character string) to be read using `tar_read()`, or a data frame directly. This flexibility is particularly useful for testing purposes, allowing you to pass data frames directly without running the entire pipeline.

However, because `tar_read()` uses non-standard evaluation (NSE) to interpret the target name, you may need to handle NSE carefully in your functions. In this section, we'll explore how `tar_read()` works, how it utilizes NSE, and how to write functions that can flexibly handle both cases.

---

### Understanding `tar_read()` and Non-Standard Evaluation

The `tar_read()` function from the `targets` package is designed to read a target from your pipeline. You can specify the target to read either as an unquoted symbol or as a character string. Internally, `tar_read()` uses NSE to capture the expression provided and resolve it to the name of the target.

**Examples of `tar_read()` Usage:**

```r
library(targets)

# Assuming you have a target named 'my_data' in your pipeline

# Using an unquoted symbol
data <- tar_read(my_data)

# Using a character string
data <- tar_read("my_data")
```

- In the first example, `my_data` is an unquoted symbol. `tar_read()` uses NSE to capture this symbol and determine the target to read.
- In the second example, `"my_data"` is a character string specifying the target name directly.

---

### Writing Functions That Accept Either Target Names or Data Frames

To create functions that can accept either a target name (to be read with `tar_read()`) or a data frame directly, you need to handle both cases within your function. One approach is to check the type of the input and act accordingly.

**Example Function Handling Both Cases:**

```r
library(targets)
library(dplyr)

process_data <- function(data) {
  if (is.character(data)) {
    # 'data' is a character string specifying the target name
    data <- tar_read(data)
  } else if (is.symbol(substitute(data))) {
    # 'data' is an unquoted symbol (target name)
    data_name <- deparse(substitute(data))
    data <- tar_read(data_name)
  }
  # Proceed with data processing
  result <- data %>%
    summarize(count = n())
  
  return(result)
}

# Usage with unquoted target name
output <- process_data(my_data)

# Usage with quoted target name (character string)
output <- process_data("my_data")

# Usage with a data frame directly (e.g., for testing)
test_data <- data.frame(x = 1:10)
output <- process_data(test_data)
```

**Explanation:**

- **Unquoted Target Name (Symbol):**
  - We use `substitute(data)` to capture the expression passed to `data`.
  - If it's a symbol (unquoted target name), we convert it to a string with `deparse(substitute(data))` and pass it to `tar_read()`.

- **Character String Target Name:**
  - If `data` is a character string, we pass it directly to `tar_read()`.

- **Data Frame Directly:**
  - If `data` is neither a character string nor a symbol, we assume it's a data frame and proceed with processing.

---

### Handling NSE with `rlang`

Alternatively, you can use the `rlang` package to handle NSE more robustly.

**Using `rlang` to Manage NSE:**

```r
library(targets)
library(dplyr)
library(rlang)

process_data <- function(data) {
  data_expr <- enquo(data)
  
  if (quo_is_symbol(data_expr)) {
    # 'data' is an unquoted symbol (target name)
    data_name <- as_name(data_expr)
    data <- tar_read(data_name)
  } else if (quo_is_string(data_expr)) {
    # 'data' is a character string
    data_name <- eval_tidy(data_expr)
    data <- tar_read(data_name)
  } else if (quo_is_call(data_expr)) {
    # 'data' is evaluated as an expression (e.g., data frame variable)
    data <- eval_tidy(data_expr)
  } else {
    stop("Invalid input: please provide a data frame or a target name.")
  }
  
  # Proceed with data processing
  result <- data %>%
    summarize(count = n())
  
  return(result)
}

# Usage with unquoted target name
output <- process_data(my_data)

# Usage with quoted target name (character string)
output <- process_data("my_data")

# Usage with a data frame directly
test_data <- data.frame(x = 1:10)
output <- process_data(test_data)
```

**Explanation:**

- **`enquo(data)`:**
  - Captures the argument passed to `data` as a quosure, which contains both the expression and its environment.

- **Interpreting `data`:**
  - **If `data` is a symbol (unquoted target name):**
    - Use `quo_is_symbol()` to check.
    - Convert to string with `as_name()` and read using `tar_read()`.
  - **If `data` is a string:**
    - Use `quo_is_string()` to check.
    - Evaluate the string and pass to `tar_read()`.
  - **If `data` is a call or expression:**
    - Use `quo_is_call()` to check.
    - Evaluate the expression with `eval_tidy()`.

- **Data Frame Handling:**
  - If `data` evaluates to a data frame, proceed with processing.
  - If `data` is invalid, throw an error.

---

### Practical Example

**Suppose you have the following target pipeline in `_targets.R`:**

```r
library(targets)

tar_option_set(packages = c("dplyr"))

list(
  tar_target(
    my_data,
    data.frame(x = rnorm(100), y = rnorm(100))
  ),
  tar_target(
    result,
    process_data(my_data)
  )
)
```

- **`my_data`**: A target that creates a data frame.
- **`result`**: A target that processes `my_data` using `process_data()`.

**In your `functions.R` file:**

```r
process_data <- function(data) {
  # Function as defined earlier
  # ...
}
```

**Testing the Function Directly:**

```r
# Load the functions
source("functions.R")

# Test with a data frame
test_df <- data.frame(x = 1:10)
process_data(test_df)

# Test with a character string target name
process_data("my_data")

# Test with an unquoted target name (assuming `my_data` exists in the environment)
process_data(my_data)
```

---

### Additional Considerations

- **Avoid Ambiguity with Explicit Arguments:**
  - To prevent confusion, you might choose to require that target names be provided as character strings.
  - Alternatively, you could have separate arguments or functions for handling data frames and target names.

- **Documentation and User Guidance:**
  - Clearly document how users should provide the `data` argument.
  - Explain the expected input types and any constraints.

- **Error Handling:**
  - Implement robust error handling to provide informative messages.
  - Use `is.data.frame()` to check if `data` is a data frame after evaluation.

---

### Simplifying with Separate Functions

If managing NSE becomes too complex, consider separating the functionality:

- **Function for Data Frames:**

  ```r
  process_data_df <- function(data) {
    data %>%
      summarize(count = n())
  }
  ```

- **Function for Target Names:**

  ```r
  process_data_target <- function(target_name) {
    data <- tar_read(target_name)
    process_data_df(data)
  }
  ```

- **Usage:**

  ```r
  # For data frames
  result <- process_data_df(test_data)

  # For target names
  result <- process_data_target("my_data")
  ```

This approach simplifies each function and makes the expected input explicit.

---

### Conclusion

Handling non-standard evaluation and functions like `tar_read()` requires careful consideration, especially when designing flexible functions. By using functions like `substitute()`, `enquo()`, and tools from the `rlang` package, you can create functions that accept both unquoted target names, character strings, and data frames.

Key points:

- **Understand how `tar_read()` handles NSE**: It can accept both symbols and character strings.
- **Use `enquo()` and `rlang` functions to manage NSE in your own functions**.
- **Provide clear documentation and error handling** to guide users in providing the correct inputs.
- **Consider separating functionality** if the NSE handling becomes too complex.

By applying these strategies, you can build robust data processing functions that integrate smoothly with the `targets` package and provide flexibility for both pipeline execution and testing.

---

**References:**

- [targets Package Documentation](https://books.ropensci.org/targets/)
- [Non-Standard Evaluation in R](https://adv-r.hadley.nz/evaluation.html)
- [rlang Package Documentation](https://rlang.r-lib.org/)
By applying these strategies, you can build robust data processing functions that integrate smoothly with the `targets` package and provide flexibility for both pipeline execution and testing.

---

**References:**

- [targets Package Documentation](https://books.ropensci.org/targets/)
- [Non-Standard Evaluation in R](https://adv-r.hadley.nz/evaluation.html)
- [rlang Package Documentation](https://rlang.r-lib.org/)